<a href="https://colab.research.google.com/github/nenaplnk/AOT_classifier/blob/main/AOT_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# распаковка rar
import os

files = [f for f in os.listdir('.') if f.endswith('.rar')]
if files:
    !apt-get install -y unrar
    for file in files:
        !unrar x "{file}" "/content/extracted/"
        print(f"Распакован: {file}")
else:
    print("RAR файлы не найдены")

In [447]:
#импорт библиотек остальных....
import json
import torch
import torch.utils.data as data
from torchvision import models
import torchvision.transforms.v2 as tfs
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
from torch.utils.data import ConcatDataset


In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [442]:
#свой датасет из распакованной rar
class AotDataset(data.Dataset):
    def __init__(self, path, train=True, transform=None):
        self.path = os.path.join(path, "train" if train else "test")
        self.transform = transform

        with open(os.path.join(self.path, "format.json"), "r") as fp:
            self.format = json.load(fp)

        self.length = 0
        self.files = []
        self.targets = torch.eye(7)

        for _dir, _target in self.format.items():
            path = os.path.join(self.path, _dir)
            list_files = os.listdir(path)
            self.length += len(list_files)
            self.files.extend(map(lambda _x: (os.path.join(path, _x), _target), list_files))

    def __getitem__(self, item):
        path_file, target = self.files[item]
        t = self.targets[target]
        img = Image.open(path_file)

        if self.transform:
            img = self.transform(img)
        return img, t

    def __len__(self):
        return self.length
w = models.ResNet18_Weights.DEFAULT # реснет(веса)
transforms = w.transforms() # реснет(tfs)

In [ ]:
model = models.resnet18(weights = w).to(device) #выбор маленькой модели реснет
model.requires_grad_(False) #замораживаем веса всей модели
model.fc = nn.Linear(512, 7).to(device) #модифицируем fc слой под количество классов перекладываем на gpu
model.fc.requires_grad_(True) #отмораживаем fc веса
model.layer4.requires_grad_(True).to(device) #разморозка последнего сверточного слоя и перекладка на gpu

In [444]:
#создание 2 объектов нашего класса Dataset для треина и теста
d_train = AotDataset(r"extracted/imgs_titans", transform = transforms)
d_test = AotDataset(r"extracted/imgs_titans", train=False, transform=transforms)
aug = tfs.Compose(
    [
        tfs.RandomRotation(degrees=15),
        transforms
    ]
)
d_train_aug = AotDataset(r"extracted/imgs_titans", transform = aug)
d_train_vmeste = ConcatDataset([d_train, d_train_aug])
d_train_train, d_train_valid =  data.random_split(d_train_vmeste, [0.7, 0.3])
#создания 2 даталоудеров для треина и теста
train_data = data.DataLoader(d_train_train, batch_size = 10, shuffle = True)
valid_data = data.DataLoader(d_train_valid, batch_size = 4, shuffle = True)
test_data = data.DataLoader(d_test, batch_size=3, shuffle=False)

In [445]:
#оптимизатор
optimizer = optim.Adam([
    {'params': model.fc.parameters(), 'lr': 0.001},
    {'params': model.layer4.parameters(), 'lr': 0.000001}
], weight_decay=0.001)
loss_func = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(55):
  epoch_loss = 0
  count = 0
  epoch_loss2 = 0
  count2 = 0
  for x_train, y_train in train_data:
    model.train()
    x_train = x_train.to(device)
    y_train = y_train.to(device)
    predict = model(x_train)
    L = loss_func(predict, y_train)
    optimizer.zero_grad()
    L.backward()
    optimizer.step()
    epoch_loss += L.item()
    count += 1
  for x_valid, y_valid in valid_data:
    model.eval()
    x_valid = x_valid.to(device)
    y_valid = y_valid.to(device)
    pr = (model(x_valid))
    L = loss_func(pr, y_valid)
    epoch_loss2 += L.item()
    count2 += 1
  print(f"epoch = {epoch} , L_mean_train = {epoch_loss/count}, L_mean_valid = {epoch_loss2/count2}")

In [453]:
s = 0
for x_test, y_test in test_data:
    model.eval()
    x_test = x_test.to(device)
    y_test = y_test.to(device)
    pr = torch.argmax(model(x_test), dim = 1)
    y = torch.argmax(y_test, dim = 1)
    s+=torch.sum(pr == y).item()
print(s/len(d_test))

0.6


In [436]:
def beauty_output(file_name:str):
  cats = ["anny", "armin", "eren", "galyart", "levi", "mikasa", "peak"]
  model.eval()
  img_test = Image.open(file_name)
  img_tfs = transforms(img_test).unsqueeze(0).to(device)
  with torch.no_grad():
    probas = torch.softmax(model(img_tfs), dim = 1)
    ans = torch.argmax(model(img_tfs), dim = 1).item()
    proba = probas[0][ans].item()
  print("Ваш результат:")
  print("__"*40)
  print(f"Наиболее вероятный персонаж : {cats[ans]}!")
  print(f"Уверенность модельки в выборе : {proba}!")
  print("Другие возможные классы:")
  lst_tst = []
  for i, prob in enumerate(probas[0]):
    lst_tst.append([cats[i], prob])
  lst_tst.sort(key = lambda x: x[1], reverse = True)
  for el in lst_tst:
    print(f"{el[0]}:{el[1]:.2%}")
  return

In [ ]:
beauty_output('твоя фотка')

In [437]:
#пример реза

Ваш результат:
________________________________________________________________________________
Наиболее вероятный персонаж : armin!
Уверенность модельки в выборе : 0.5966334342956543!
Другие возможные классы:
armin:59.66%
peak:25.50%
mikasa:12.19%
eren:1.73%
anny:0.82%
levi:0.05%
galyart:0.05%


In [438]:
data_st_d = {
    "loss":loss_func.state_dict(),
    "opt":optimizer.state_dict(),
    "model":model.state_dict()
}

In [439]:
torch.save(data_st_d, "aot_state_dict_fff.tar")

In [451]:
t = torch.load("aot_state_dict_fff.tar")

In [448]:
model.load_state_dict(t['model'])

<All keys matched successfully>